In [ ]:
import os
import torch.nn as nn
import torch.optim as optim
from myconfig.Config import *
from trans1utils.DataGenerater import *
from transformer.Transformer import Transformer

In [ ]:
vocab,vocab_size = vocab_config()
hla_max_len,pep_max_len,tcr_max_len,hla_pep_concat_len,pep_tcr_concat_len = data_config()
d_model, d_ff, d_k, d_v, n_layers,n_heads,epochs, batch_size, threshold, dropout_rate, max_len,device = model_config()
seed = run_config()

train_hla_pep_loader,val_hla_pep_loader = hla_pep_data_loader('hla_pep_dataset',vocab,hla_max_len,pep_max_len,batch_size,0.8)
train_pep_tcr_loader,val_pep_tcr_loader = pep_tcr_data_loader('pep_tcr_dataset',vocab,pep_max_len,tcr_max_len,batch_size,0.8)

In [ ]:
print('--------------构建模型--------------')
model = Transformer(vocab_size,hla_pep_concat_len=hla_pep_concat_len,pep_tcr_concat_len=pep_tcr_concat_len).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = 1e-3)

In [ ]:
train_metrics_list, val_metrics_list = [], []

print('------------训练和验证模型------------')
dir_saver = './model/'
path_saver = './model/model_layer{}_head{}.pkl'.format(n_layers, n_heads)
print('dir_saver: ', dir_saver)
print('path_saver: ', path_saver)
metric_best, ep_best,time_train = 0, -1, 0
for epoch in range(1, 5):
    
    ys_train,loss_train_list,metrics_train = model.train_per_epoch(train_hla_pep_loader,epoch,epochs,criterion,optimizer,num_group=1,threshold=threshold,metrics_print_=True)
    
    ys_val, loss_val_list, metrics_val = model.eval_per_epoch(val_hla_pep_loader, epoch, epochs,criterion,num_group=1,threshold=threshold,metrics_print_=True)

    metrics_ep_avg = sum(metrics_val[:4])/4
    print("------------保存指标: " , metrics_ep_avg)

    if metrics_ep_avg > metric_best:
        metric_best, ep_best = metrics_ep_avg, epoch
        if not os.path.exists(dir_saver):
            os.makedirs(dir_saver)
        print('Best epoch = {} | Metric_best_avg = {:.4f}--------'.format(ep_best, metric_best))
        print('Saving model Path saver: {} --------'.format(path_saver))
        torch.save(model.eval().state_dict(), path_saver)

In [ ]:
train_metrics_list, val_metrics_list = [], []

print('--------------构建模型--------------')


print('------------训练和验证模型------------')
dir_saver = './model/'
path_saver = './model/model_layer{}_head{}.pkl'.format(n_layers, n_heads)
print('dir_saver: ', dir_saver)
print('path_saver: ', path_saver)
metric_best, ep_best,time_train = 0, -1, 0
for epoch in range(1, 5):
    
    ys_train,loss_train_list,metrics_train = model.train_per_epoch(train_pep_tcr_loader,epoch,epochs,criterion,optimizer,num_group=2,threshold=threshold,metrics_print_=True)
    
    ys_val, loss_val_list, metrics_val = model.eval_per_epoch(val_pep_tcr_loader, epoch, epochs,criterion,num_group=2,threshold=threshold,metrics_print_=True)

    metrics_ep_avg = sum(metrics_val[:4])/4
    print("------------保存指标: " , metrics_ep_avg)

    if metrics_ep_avg > metric_best:
        metric_best, ep_best = metrics_ep_avg, epoch
        if not os.path.exists(dir_saver):
            os.makedirs(dir_saver)
        print('Best epoch = {} | Metric_best_avg = {:.4f}--------'.format(ep_best, metric_best))
        print('Saving model Path saver: {} --------'.format(path_saver))
        torch.save(model.eval().state_dict(), path_saver)